**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [47]:
import pandas as pd

In [48]:
geo_data = {
                'Центр': ['москва', 'тула', 'ярославль'],
                'Северо-Запад': ['петербург', 'псков', 'мурманск'],
                'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }

In [49]:
geo_data_df = pd.DataFrame(geo_data)

geo_data_values = []
for keys, values in geo_data.items():
    geo_data_values += values

['москва',
 'тула',
 'ярославль',
 'петербург',
 'псков',
 'мурманск',
 'владивосток',
 'сахалин',
 'хабаровск']

In [50]:
b = 'вконтакте псков'

In [52]:
keywords = pd.read_csv('keywords.csv')
keywords.head(2)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309


In [62]:
def geo_clasificator (row):
    city = 'undefined'
    for value in geo_data_values:
        if value in row['keyword']:
            city = value
            for keys, values in geo_data.items(): 
                if city in values:
                    return keys
    if city == 'undefined':
        return city

In [63]:
keywords['region'] = keywords.apply(geo_clasificator, axis=1)

In [64]:
keywords[(keywords['keyword'] == 'владивосток')]

,keyword,shows,region
5858,владивосток,43038,Дальний Восток


In [65]:
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('C:/Users/anton/Documents/NETOLOGY/Courses/Python/Homework/ml-latest-small/ratings.csv')

In [3]:
def class_rate (row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [4]:
ratings['class'] = ratings.groupby('movieId').mean().reset_index().apply(class_rate, axis=1)

In [5]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,средний рейтинг
4,1,50,5.0,964982931,средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [258]:
import pandas as pd
import time
from datetime import date
from datetime import datetime

In [259]:
ratings = pd.read_csv('C:/Users/anton/Documents/NETOLOGY/Courses/Python/Homework/ml-latest-small/ratings.csv')

In [260]:
ratings_filtered = ratings.groupby('userId').count().reset_index()

In [261]:
ratings_filtered = ratings_filtered[ratings_filtered['rating']>100]

In [262]:
ratings_filtered_agg = ratings[ratings['userId'].isin(ratings_filtered['userId'].tolist())]

In [263]:
ratings_filtered_agg = ratings[ratings['userId'].isin(ratings_filtered['userId'].tolist())].groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()

In [264]:
def date_convert (row, column_name):
    date_convert_tmp = str(pd.Timestamp(row[column_name], unit='s'))
    return datetime.strptime(date_convert_tmp, '%Y-%m-%d %H:%M:%S')

In [265]:
def subtraction_date (row, column_name1, column_name2):
    subtraction_tmp = ratings_filtered_agg[column_name2] - ratings_filtered_agg[column_name1]
    return subtraction_tmp.days

In [266]:
# def date_convert (row, column_name):
#     return datetime.fromtimestamp(column_name)

In [270]:
ratings_filtered_agg['min'] = ratings_filtered_agg.apply(date_convert, column_name = 'min', axis=1)
ratings_filtered_agg['max'] = ratings_filtered_agg.apply(date_convert, column_name = 'max', axis=1)
ratings_filtered_agg['Lifetime'] = ratings_filtered_agg['max'] - ratings_filtered_agg['min']
# ratings_filtered_agg['Lifetime'] = ratings_filtered_agg.apply(subtraction_date, column_name2 = 'max', column_name1 = 'min', axis=1)

In [271]:
ratings_filtered_agg.head()

,userId,min,max,Lifetime
0,1,2000-07-30 18:08:19,2000-08-08 07:27:42,8 days 13:19:23
1,4,1999-12-13 09:47:08,2001-12-05 17:49:02,723 days 08:01:54
2,6,1996-10-17 11:51:49,1996-10-17 12:55:15,0 days 01:03:26
3,7,2005-01-25 06:43:36,2007-04-10 05:08:51,804 days 22:25:15
4,10,2016-02-12 18:25:53,2016-02-16 10:41:15,3 days 16:15:22


In [272]:
Lifetime_mean = ratings_filtered_agg['Lifetime'].mean()

In [274]:
Lifetime_mean.days

517

In [ ]:
# Почему datetime отображается в столбцах в формате даты, а не в формате datetime.datetime(2018, 5, 9, 9, 0)?
# Как вывести в столбце количесвто дней в формате int? Никак не смог избавиться от времени.

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [429]:
import pandas as pd
import time
from datetime import date
from datetime import datetime

In [430]:
years = [str(i) for i in range(1950,2011)]

In [431]:
movies = pd.read_csv('C:/Users/anton/Documents/NETOLOGY/Courses/Python/Homework/ml-latest-small/movies.csv')
ratings = pd.read_csv('C:/Users/anton/Documents/NETOLOGY/Courses/Python/Homework/ml-latest-small/ratings.csv')

In [432]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [433]:
def production_year (row, b):
    try:
        year = row[b].split('(')[1].split(')')[0]
        if year in years:
            return year
        else:
            return '1900'
    except IndexError:
        return '1900'

In [441]:
movies['year'] = movies.apply(production_year, b = 'title', axis=1)

In [442]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1900
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [452]:
joined.groupby('year').mean()['rating'].reset_index().sort_values('rating', ascending=False).head()

,year,rating
8,1957,4.022472
5,1954,4.000000
13,1962,3.985477
3,1952,3.957447
15,1964,3.952830
